In [4]:
import pandas as pd
import numpy as np
import cv2
import csv
from scipy.stats import pearsonr
from random import randint
from copy import deepcopy

In [5]:
train_faces_csv = r"C:\Users\ABHINAV\Downloads\mlware23\dataset\train\train_faces.csv"
train_landmarks_csv = r"C:\Users\ABHINAV\Downloads\mlware23\dataset\train\train_landmarks.csv"
train_faces = r"C:\Users\ABHINAV\Downloads\mlware23\dataset\train\faces"
train_landamrks = r"C:\Users\ABHINAV\Downloads\mlware23\dataset\train\landmarks"
sample_pred = r"C:\Users\ABHINAV\Downloads\mlware23\dataset\sample_prediction.csv"
test_img = r"C:\Users\ABHINAV\Downloads\mlware23\dataset\test\test_img"

In [6]:
df = pd.read_csv(sample_pred)
img_name = df.iloc[0][0]
print(img_name)
img_path = test_img + chr(92) + img_name
print(img_path)
img = cv2.imread(img_path)
print(img.shape)
cv2.imshow("img", img)
cv2.waitKey(0)

xWVL9VlQPD.jpg
C:\Users\ABHINAV\Downloads\mlware23\dataset\test\test_img\xWVL9VlQPD.jpg
(300, 300, 3)


-1

Function to obtain corners (top left and bottom right) of individual tiles of the image

In [7]:
def get_corners(img):
    keys = []
    values = []
    for i in range(6):
        for j in range(6):
            keys.append(tuple([i,j]))
            x1 = (50*i)
            y1 = (50*j)
            x2 = (50*(i+1))
            y2 = (50*(j+1))
            if i == 0:
                x1 = 0
            if j == 0:
                y1 = 0
            values.append([[x1,y1],[x2,y2]])
    return dict(zip(keys, values))

In [8]:
dictionary = get_corners(img)
print(dictionary)

{(0, 0): [[0, 0], [50, 50]], (0, 1): [[0, 50], [50, 100]], (0, 2): [[0, 100], [50, 150]], (0, 3): [[0, 150], [50, 200]], (0, 4): [[0, 200], [50, 250]], (0, 5): [[0, 250], [50, 300]], (1, 0): [[50, 0], [100, 50]], (1, 1): [[50, 50], [100, 100]], (1, 2): [[50, 100], [100, 150]], (1, 3): [[50, 150], [100, 200]], (1, 4): [[50, 200], [100, 250]], (1, 5): [[50, 250], [100, 300]], (2, 0): [[100, 0], [150, 50]], (2, 1): [[100, 50], [150, 100]], (2, 2): [[100, 100], [150, 150]], (2, 3): [[100, 150], [150, 200]], (2, 4): [[100, 200], [150, 250]], (2, 5): [[100, 250], [150, 300]], (3, 0): [[150, 0], [200, 50]], (3, 1): [[150, 50], [200, 100]], (3, 2): [[150, 100], [200, 150]], (3, 3): [[150, 150], [200, 200]], (3, 4): [[150, 200], [200, 250]], (3, 5): [[150, 250], [200, 300]], (4, 0): [[200, 0], [250, 50]], (4, 1): [[200, 50], [250, 100]], (4, 2): [[200, 100], [250, 150]], (4, 3): [[200, 150], [250, 200]], (4, 4): [[200, 200], [250, 250]], (4, 5): [[200, 250], [250, 300]], (5, 0): [[250, 0], [300

test

In [9]:
lis = dictionary[tuple([0, 1])]
print(lis)
print(lis[1][0])
#crop = img[0:49, 49:99]
crop = img[lis[0][0]:lis[1][0], lis[0][1]:lis[1][1]]
cv2.imshow("img", crop)
cv2.waitKey(0)
print(crop.shape)

[[0, 50], [50, 100]]
50
(50, 50, 3)


Function to perform pixel invariant transpose on an image

In [10]:
def img_transpose(img):
    arr = np.zeros(img.shape)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            arr[i][j] = img[j][i]
    return arr.astype(int)

Function to obtain the top , bottom, left and right edges of a tile, returns a dictionary

In [11]:
def get_edge_vector(img, dictionary):
    keys = []
    values = []

    for i in range(6):
        for j in range(6):
            keys.append(tuple([i, j]))
            lis = dictionary[tuple([i, j])]
            crop = img[lis[0][0]:lis[1][0], lis[0][1]: lis[1][1]]
            #print(crop.shape)
            crop_t = img_transpose(crop)
            top = crop[0]
            bottom = crop[-1]
            left = crop_t[0]
            right = crop_t[-1]
            values.append([top, bottom, left, right])
    return(dict(zip(keys, values)))


Function for finding euclidean distance between two edges fo tiles

In [12]:
def edge_dist(e1, e2):
    dist = 0
    for i in range(e1.shape[0]):
        d = (((e1[i][0]-e2[i][0])**2)+((e1[i][1]-e2[i][1])**2)+((e1[i][2]-e2[i][2])**2))**0.5
        dist = dist+d
        d = 0
    return int(dist/50)

Function for finding pearson correlation between two edges of the tiles

In [13]:
def pearson_line(e1, e2):
    ch_r1 = []
    ch_g1 = []
    ch_b1 = []
    ch_r2 = []
    ch_b2 = []
    ch_g2 = []
    for i in range(50):
        ch_r1.append(e1[i][0])
        ch_g1.append(e1[i][1])
        ch_b1.append(e1[i][2])
        ch_r2.append(e2[i][0])
        ch_g2.append(e2[i][1])
        ch_b2.append(e2[i][2])
    corr_r, _ = pearsonr(ch_r1, ch_r2)
    corr_b, _ = pearsonr(ch_b1, ch_b2)
    corr_g, _ = pearsonr(ch_g1, ch_g2)
    sum = corr_r+corr_g+corr_b
    return sum

In [14]:
edge_dict = get_edge_vector(img, dictionary)
#print(edge_dist(edge_dict[(0, 0)][0], edge_dict[(0, 0)][0]))

Euclediean distance based nearest tile detection

In [12]:
"""comb = []
for i in range(6):
    for j in range(6):
        comb.append((i, j))
#print(comb)
#print(edge_dict)
def get_positions(edge_dict, comb):
    keys = []
    values = []
    for item in comb:
        keys.append(item)
        closest_ij = []
        min_top = 10000000
        min_bottom = 10000000
        min_left = 10000000
        min_right = 10000000
        top_ij = ()
        bottom_ij = ()
        left_ij = ()
        right_ij = ()
        for i in range(6):
            for j in range(6):
            
                top1 = edge_dict[item][0]
                bottom1 = edge_dict[item][1]
                left1 = edge_dict[item][2]
                right1 = edge_dict[item][3]
                if item !=(i, j):
                    top2 = edge_dict[(i, j)][0]
                    bottom2 = edge_dict[(i, j)][1]
                    left2 = edge_dict[(i, j)][2]
                    right2 = edge_dict[(i, j)][3]
                    if edge_dist(top1, bottom2)<min_top:
                        top_ij = (i, j)
                        min_top = edge_dist(top1, bottom2)
                    if edge_dist(bottom1, top2)<min_bottom:
                        bottom_ij = (i, j)
                        min_bottom = edge_dist(bottom1, top2)
                    if edge_dist(left1, right2)<min_left:
                        left_ij = (i, j)
                        min_left = edge_dist(left1, right2)
                    if edge_dist(right1, left2)<min_right:
                        right_ij = (i, j)
                        min_right = edge_dist(right1, left2)
        #closest_ij.append()
        values.append([top_ij, bottom_ij, left_ij, right_ij])
    return dict(zip(keys, values))
"""


'comb = []\nfor i in range(6):\n    for j in range(6):\n        comb.append((i, j))\n#print(comb)\n#print(edge_dict)\ndef get_positions(edge_dict, comb):\n    keys = []\n    values = []\n    for item in comb:\n        keys.append(item)\n        closest_ij = []\n        min_top = 10000000\n        min_bottom = 10000000\n        min_left = 10000000\n        min_right = 10000000\n        top_ij = ()\n        bottom_ij = ()\n        left_ij = ()\n        right_ij = ()\n        for i in range(6):\n            for j in range(6):\n            \n                top1 = edge_dict[item][0]\n                bottom1 = edge_dict[item][1]\n                left1 = edge_dict[item][2]\n                right1 = edge_dict[item][3]\n                if item !=(i, j):\n                    top2 = edge_dict[(i, j)][0]\n                    bottom2 = edge_dict[(i, j)][1]\n                    left2 = edge_dict[(i, j)][2]\n                    right2 = edge_dict[(i, j)][3]\n                    if edge_dist(top1, bo

Pearson correlation based nearest tile detection 

In [15]:
comb = []
for i in range(6):
    for j in range(6):
        comb.append((i, j))
def get_positions1(edge_dict, comb):
    keys = []
    values = []
    for item in comb:
        keys.append(item)
        closest_ij = []
        min_top = -4
        min_bottom = -4
        min_left = -4
        min_right = -4
        top_ij = ()
        bottom_ij = ()
        left_ij = ()
        right_ij = ()
        for i in range(6):
            for j in range(6):
            
                top1 = edge_dict[item][0]
                bottom1 = edge_dict[item][1]
                left1 = edge_dict[item][2]
                right1 = edge_dict[item][3]
                if item !=(i, j):
                    top2 = edge_dict[(i, j)][0]
                    bottom2 = edge_dict[(i, j)][1]
                    left2 = edge_dict[(i, j)][2]
                    right2 = edge_dict[(i, j)][3]
                    if pearson_line(top1, bottom2)>min_top:
                        top_ij = (i, j)
                        min_top = pearson_line(top1, bottom2)
                    if pearson_line(bottom1, top2)>min_bottom:
                        bottom_ij = (i, j)
                        min_bottom = pearson_line(bottom1, top2)
                    if pearson_line(left1, right2)>min_left:
                        left_ij = (i, j)
                        min_left = pearson_line(left1, right2)
                    if pearson_line(right1, left2)>min_right:
                        right_ij = (i, j)
                        min_right = pearson_line(right1, left2)
        #closest_ij.append()
        lis = [top_ij, bottom_ij, left_ij, right_ij]
        for i in range(len(lis)):
            if lis[i] == ():
                lis[i] = (-1, -1)
        values.append(lis)
    return dict(zip(keys, values))

        

In [24]:
def pixel_diff(p1, p2):
    diff = 0
    treshold = 10
    for i in range(p1.shape[0]):
        diff += abs(int(p1[i] - int(p2[i])))
    return False if diff < treshold else True

def side_difference(e1, e2):
    difference = 0
    for i in range(e1.shape[0]):
        difference += 1 if pixel_diff(e1[i], e2[i]) else 0
    return difference

pixel distance method

In [25]:
def get_positions2(edge_dict, comb):
    keys = []
    values = []
    for item in comb:
            keys.append(item)
            closest_ij = []
            min_top = -4
            min_bottom = -4
            min_left = -4
            min_right = -4
            top_ij = ()
            bottom_ij = ()
            left_ij = ()
            right_ij = ()
            for i in range(6):
                for j in range(6):
            
                    top1 = edge_dict[item][0]
                    bottom1 = edge_dict[item][1]
                    left1 = edge_dict[item][2]
                    right1 = edge_dict[item][3]
                    if item !=(i, j):
                        top2 = edge_dict[(i, j)][0]
                        bottom2 = edge_dict[(i, j)][1]
                        left2 = edge_dict[(i, j)][2]
                        right2 = edge_dict[(i, j)][3]
                        if side_difference(top1, bottom2)>min_top:
                            top_ij = (i, j)
                            min_top = side_difference(top1, bottom2)
                        if side_difference(bottom1, top2)>min_bottom:
                            bottom_ij = (i, j)
                            min_bottom = side_difference(bottom1, top2)
                        if side_difference(left1, right2)>min_left:
                            left_ij = (i, j)
                            min_left = side_difference(left1, right2)
                        if side_difference(right1, left2)>min_right:
                            right_ij = (i, j)
                            min_right = side_difference(right1, left2)
            #closest_ij.append()
            lis = [top_ij, bottom_ij, left_ij, right_ij]
            for i in range(len(lis)):
                if lis[i] == ():
                    lis[i] = (-1, -1)
            values.append(lis)
    return dict(zip(keys, values))
            
            

In [26]:
comb = []
for i in range(6):
    for j in range(6):
        comb.append((i, j))
#closest_dict = get_positions(edge_dict, comb)
closest_dict1 = get_positions1(edge_dict, comb)
#print((closest_dict[(0, 0)]))
closest_dict2 = get_positions2(edge_dict, comb)

In [27]:
print(closest_dict1)

{(0, 0): [(1, 4), (4, 2), (2, 2), (0, 1)], (0, 1): [(3, 3), (3, 1), (0, 0), (2, 5)], (0, 2): [(3, 2), (2, 3), (1, 4), (4, 4)], (0, 3): [(2, 3), (2, 2), (5, 0), (0, 0)], (0, 4): [(2, 4), (4, 5), (1, 5), (1, 5)], (0, 5): [(1, 2), (5, 0), (2, 4), (5, 3)], (1, 0): [(2, 3), (1, 1), (0, 3), (2, 5)], (1, 1): [(2, 2), (3, 1), (5, 4), (3, 1)], (1, 2): [(5, 2), (2, 2), (5, 5), (1, 4)], (1, 3): [(1, 4), (1, 2), (3, 3), (4, 1)], (1, 4): [(4, 4), (0, 0), (1, 2), (0, 2)], (1, 5): [(1, 4), (2, 3), (1, 0), (0, 4)], (2, 0): [(3, 1), (3, 5), (4, 3), (2, 1)], (2, 1): [(3, 5), (2, 3), (0, 0), (5, 4)], (2, 2): [(1, 2), (3, 2), (5, 0), (0, 0)], (2, 3): [(1, 4), (5, 2), (3, 3), (4, 0)], (2, 4): [(1, 4), (4, 4), (3, 2), (0, 5)], (2, 5): [(1, 3), (4, 3), (0, 1), (4, 0)], (3, 0): [(3, 2), (4, 0), (4, 1), (5, 4)], (3, 1): [(5, 3), (2, 0), (3, 2), (1, 2)], (3, 2): [(2, 2), (0, 2), (5, 3), (2, 4)], (3, 3): [(3, 5), (0, 1), (5, 5), (4, 0)], (3, 4): [(5, 1), (2, 2), (5, 2), (3, 2)], (3, 5): [(2, 0), (3, 3), (2, 4), 

In [28]:
print(closest_dict2)

{(0, 0): [(0, 1), (0, 1), (0, 2), (0, 1)], (0, 1): [(0, 0), (0, 0), (0, 0), (0, 2)], (0, 2): [(0, 0), (0, 1), (0, 1), (0, 0)], (0, 3): [(0, 0), (0, 0), (0, 0), (0, 0)], (0, 4): [(0, 0), (0, 0), (0, 0), (0, 0)], (0, 5): [(0, 0), (0, 1), (0, 0), (0, 0)], (1, 0): [(0, 0), (0, 0), (0, 0), (0, 0)], (1, 1): [(0, 0), (0, 0), (0, 1), (0, 1)], (1, 2): [(0, 0), (0, 0), (0, 0), (0, 0)], (1, 3): [(0, 1), (0, 0), (0, 0), (0, 0)], (1, 4): [(0, 0), (0, 1), (0, 0), (0, 1)], (1, 5): [(0, 0), (0, 0), (0, 0), (0, 0)], (2, 0): [(0, 0), (0, 0), (0, 0), (0, 0)], (2, 1): [(0, 0), (0, 0), (0, 0), (0, 0)], (2, 2): [(0, 0), (0, 1), (0, 2), (0, 2)], (2, 3): [(0, 0), (0, 0), (0, 0), (0, 0)], (2, 4): [(0, 0), (0, 0), (0, 0), (0, 0)], (2, 5): [(0, 0), (0, 0), (0, 0), (0, 1)], (3, 0): [(0, 0), (0, 1), (0, 1), (0, 1)], (3, 1): [(0, 0), (0, 0), (0, 1), (0, 0)], (3, 2): [(0, 0), (0, 0), (0, 0), (0, 1)], (3, 3): [(0, 1), (0, 2), (0, 0), (0, 1)], (3, 4): [(0, 0), (0, 0), (0, 0), (0, 0)], (3, 5): [(0, 0), (0, 0), (0, 0), 

Function to obtain neighbouring tiles, if the value corresponding to a position is (-1, -1) then the tile in that position is uncertain/ a corner tile

In [31]:
def check_dict(closest_dict, comb):
    d = closest_dict
    keys = comb
    values = []
    for item in keys:
        final_neighbour = []
        neighbours = closest_dict[item]
        neighbour_top = neighbours[0]
        neighbour_bottom = neighbours[1]
        neighbour_left = neighbours[2]
        neighbour_right = neighbours[3]

        if neighbour_top != (-1,-1):
            if closest_dict[neighbour_top][1] == item:
                final_neighbour.append(neighbour_top)
            else:
                final_neighbour.append((-1,-1))
        else:
            final_neighbour.append((-1,-1))

        if neighbour_bottom != (-1,-1):
            if closest_dict[neighbour_bottom][0] == item:
                final_neighbour.append(neighbour_bottom)
            else:
                final_neighbour.append((-1,-1))
        else:
            final_neighbour.append((-1,-1))
        
        if neighbour_left != (-1,-1):
            if closest_dict[neighbour_left][3] == item:
                final_neighbour.append(neighbour_left)
            else:
                final_neighbour.append((-1,-1))
        else:
            final_neighbour.append((-1,-1))
        
        if neighbour_right != (-1,-1):
            if closest_dict[neighbour_right][2] == item:
                final_neighbour.append(neighbour_right)
            else:
                final_neighbour.append((-1,-1))
        else:
            final_neighbour.append((-1,-1))
        
        values.append(final_neighbour)
    return(dict(zip(keys, values)))



In [32]:
final_dict = check_dict(closest_dict1, comb)
print(final_dict)

{(0, 0): [(1, 4), (4, 2), (2, 2), (0, 1)], (0, 1): [(3, 3), (-1, -1), (0, 0), (2, 5)], (0, 2): [(3, 2), (-1, -1), (1, 4), (4, 4)], (0, 3): [(-1, -1), (-1, -1), (-1, -1), (-1, -1)], (0, 4): [(-1, -1), (-1, -1), (1, 5), (-1, -1)], (0, 5): [(-1, -1), (5, 0), (2, 4), (-1, -1)], (1, 0): [(-1, -1), (-1, -1), (-1, -1), (-1, -1)], (1, 1): [(-1, -1), (-1, -1), (-1, -1), (-1, -1)], (1, 2): [(5, 2), (2, 2), (-1, -1), (1, 4)], (1, 3): [(-1, -1), (-1, -1), (-1, -1), (-1, -1)], (1, 4): [(-1, -1), (0, 0), (1, 2), (0, 2)], (1, 5): [(-1, -1), (-1, -1), (-1, -1), (0, 4)], (2, 0): [(3, 1), (3, 5), (-1, -1), (-1, -1)], (2, 1): [(-1, -1), (-1, -1), (-1, -1), (-1, -1)], (2, 2): [(1, 2), (3, 2), (-1, -1), (0, 0)], (2, 3): [(-1, -1), (5, 2), (-1, -1), (-1, -1)], (2, 4): [(-1, -1), (4, 4), (3, 2), (0, 5)], (2, 5): [(-1, -1), (4, 3), (0, 1), (-1, -1)], (3, 0): [(-1, -1), (4, 0), (4, 1), (-1, -1)], (3, 1): [(-1, -1), (2, 0), (-1, -1), (-1, -1)], (3, 2): [(2, 2), (0, 2), (5, 3), (2, 4)], (3, 3): [(3, 5), (0, 1), 

check from here

In [35]:
def get_ij(node, matrix):
    #id = [-5,-5]
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if node[0]==matrix[i][j][0] and node[1]==matrix[i][j][1]:
                id = [i, j]
                return id
                break
    #return id

def expand_node(node, dictionary ,matrix, in_matrix, not_exp):
    values = dictionary[node]
    id = get_ij(node, matrix)
    #print(node)
    #print(id)
    #print(values)
    x = id[0]
    y = id[1]
    if x!=-5 and y!=-5:
        if values[0]!=(-1,-1):
                matrix[x-1][y] = values[0]
                if values[0] not in in_matrix:
                    in_matrix.append(values[0])
        if values[1]!=(-1,-1):
                matrix[x+1][y] = values[1]
                if values[1] not in in_matrix:
                    in_matrix.append(values[1])
        if values[2]!=(-1,-1):
                matrix[x][y-1] = values[2]
                if values[2] not in in_matrix:
                    in_matrix.append(values[2])
        if values[3]!=(-1,-1):
                matrix[x][y+1] = values[3]
                if values[3] not in in_matrix:
                    in_matrix.append(values[3])
        not_exp.remove(node)
        print("expanded: ", node)
    else:
        not_exp.remove(node)
        #unsure.append[node]

def set_priority(in_matrix, not_exp):
    x = ()
    for node in not_exp:
        if node in in_matrix:
            x = node
            break
        
    return x        


"""while len(not_exp)!=0:
    node = set_priority(in_matrix, not_exp)
    expand_node(node, dictionary, matrix, in_matrix, not_exp)
    print(1)"""


'while len(not_exp)!=0:\n    node = set_priority(in_matrix, not_exp)\n    expand_node(node, dictionary, matrix, in_matrix, not_exp)\n    print(1)'

In [ ]:
matrix = np.empty(shape=(100,100,2))
matrix.fill(-2)

not_exp = deepcopy(comb)

start = comb[0]
matrix[50][50] = start
in_matrix = [start]
#unsure = []
expand_node(start, final_dict, matrix, in_matrix, not_exp)
#print(len(not_exp))
i = 1
while len(not_exp)!=0:
    node = set_priority(in_matrix, not_exp)
    print(node, get_ij(node ,matrix))
    if node == ():
        break
    print(in_matrix)
    print(not_exp)
    #print(node)
    expand_node(node, final_dict, matrix, in_matrix, not_exp)
    print(i)
    i = i+1

In [20]:
def find_start(matrix):
    x = 0
    y = 0
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if matrix[i][j][0]!=-2 and matrix[i][j][1]!=-2:
                x = i
                y = j
                return [x,y]
                break


def find_end(matrix):
    x = 0
    y = 0
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if matrix[i][j][0]!=-2 and matrix[i][j][1]!=-2:
                x = i
                y = j
    return [x,y]


x = find_end(matrix)[0]
y = find_end(matrix)[1]
final_matrix = matrix[x-5:x+1,y-5:y+1]
#print(final_matrix.shape)
#print(final_matrix[0])
#print(matrix[50][53])

copy = deepcopy(comb)
#print(copy)
#not_present = []
for i in range(final_matrix.shape[0]):
    for j in range(final_matrix.shape[1]):
        if (final_matrix[i][j][0], final_matrix[i][j][1]) in copy:
            copy.remove((final_matrix[i][j][0], final_matrix[i][j][1]))
print(copy)
x = 0
for i in range(final_matrix.shape[0]):
    for j in range(final_matrix.shape[1]):
        if final_matrix[i][j][0] < 0 and final_matrix[i][j][1] < 0:
            final_matrix[i][j] = copy[x]
            x = x+1
print(final_matrix[5])

[(0, 3), (1, 3), (2, 2)]
[[5. 4.]
 [4. 0.]
 [3. 4.]
 [1. 1.]
 [2. 5.]
 [1. 5.]]


In [21]:
print(get_ij((2,2), final_matrix))

[4, 3]


In [22]:
def fetch_positions(final_matrix, comb):
    final_pos = []
    for item in comb:
        final_pos.append(get_ij(item, final_matrix))
    return final_pos

print(fetch_positions(final_matrix, comb))
print(comb)            

[[0, 4], [0, 0], [4, 4], [1, 5], [1, 0], [1, 4], [3, 4], [5, 3], [0, 2], [1, 1], [5, 5], [2, 5], [2, 0], [0, 3], [4, 3], [4, 1], [2, 2], [0, 5], [2, 1], [4, 0], [2, 4], [3, 5], [1, 2], [1, 3], [5, 4], [3, 2], [4, 2], [5, 0], [3, 0], [3, 1], [4, 5], [2, 3], [5, 2], [0, 1], [3, 3], [5, 1]]
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5)]
